# Modeling #

## Import APIs ##

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

import wfdb
import ast

## Load data ##

### Metadata ###

In [2]:
ptbxl_df = pd.read_csv('./cleaned_data/cleaned_ptbxl_metadata.csv', index_col='ecg_id')

In [3]:
ptbxl_df.head()

,age,sex,device,scp_codes,validated_by_human,diagnostic_superclass,strat_fold,filename_lr,filename_hr
ecg_id,,,,,,,,,
1,56.0,1,CS-12 E,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",True,['NORM'],3,records100/00000/00001_lr,records500/00000/00001_hr
2,19.0,0,CS-12 E,"{'NORM': 80.0, 'SBRAD': 0.0}",True,['NORM'],2,records100/00000/00002_lr,records500/00000/00002_hr
3,37.0,1,CS-12 E,"{'NORM': 100.0, 'SR': 0.0}",True,['NORM'],5,records100/00000/00003_lr,records500/00000/00003_hr
4,24.0,0,CS-12 E,"{'NORM': 100.0, 'SR': 0.0}",True,['NORM'],3,records100/00000/00004_lr,records500/00000/00004_hr
5,19.0,1,CS-12 E,"{'NORM': 100.0, 'SR': 0.0}",True,['NORM'],4,records100/00000/00005_lr,records500/00000/00005_hr


In [4]:
metadata = ptbxl_df.loc[:, ['age', 'sex', 'device', 'validated_by_human']].copy()
metadata.head()

,age,sex,device,validated_by_human
ecg_id,,,,
1,56.0,1,CS-12 E,True
2,19.0,0,CS-12 E,True
3,37.0,1,CS-12 E,True
4,24.0,0,CS-12 E,True
5,19.0,1,CS-12 E,True


### Waveform data ###

In [5]:
waveform_data = []
for idx in ptbxl_df.index:
    record_path = ptbxl_df.loc[idx]['filename_hr']
    waveform_df = pd.read_csv('./cleaned_data/waveform_data/' + record_path + '.csv', index_col='Time (s)')
    waveform_data.append(waveform_df)
waveform_data = np.array(waveform_data)
waveform_data.shape

(21799, 1000, 12)

In [6]:
# waveform_data = np.load("./cleaned_data/waveform_np.npy")

## Create recommended train-test split ##

This recommended train-test split code was obtained from the downloaded folder with the dataset: https://physionet.org/content/ptb-xl/1.0.3/.

In [7]:
# Split data into train and test
test_fold = 10

# Train
waveform_train = waveform_data[np.where(ptbxl_df.strat_fold != test_fold)]
metadata_train = metadata[ptbxl_df.strat_fold != test_fold]
y_train = ptbxl_df[ptbxl_df.strat_fold != test_fold].diagnostic_superclass

# Test
waveform_test = waveform_data[np.where(ptbxl_df.strat_fold == test_fold)]
metadata_test = metadata[ptbxl_df.strat_fold == test_fold]
y_test = ptbxl_df[ptbxl_df.strat_fold == test_fold].diagnostic_superclass

## Normalize waveform data ##

In [8]:
# Code generated from Bing Copilot
normalized_waveform_train = np.empty_like(waveform_train)

for i in range(waveform_train.shape[0]):
    for j in range(waveform_train.shape[2]):
        min_val = np.min(waveform_train[i, :, j])
        max_val = np.max(waveform_train[i, :, j])

        if max_val == min_val:
            normalized_waveform_train[i, :, j] = 0
        else:
            normalized_waveform_train[i, :, j] = (waveform_train[i, :, j] - min_val) / (max_val - min_val)

In [9]:
normalized_waveform_train[0]

array([[0.15903308, 0.20707071, 0.73684211, ..., 0.31463415, 0.07619048,
        0.10942761],
       [0.15903308, 0.20707071, 0.73684211, ..., 0.31463415, 0.07619048,
        0.10942761],
       [0.15903308, 0.20707071, 0.73684211, ..., 0.31463415, 0.07619048,
        0.10942761],
       ...,
       [0.2913486 , 0.20538721, 0.50877193, ..., 0.3597561 , 0.1031746 ,
        0.27609428],
       [0.22137405, 0.17845118, 0.59210526, ..., 0.35487805, 0.1       ,
        0.26262626],
       [0.22264631, 0.19191919, 0.60745614, ..., 0.35      , 0.1       ,
        0.24915825]])

## Normalize and one-hot encode metadata ##

In [46]:
# Make tensors
transformers = [
    ('num', StandardScaler(), ['age']),
    ('cat', OneHotEncoder(), ['sex', 'device', 'validated_by_human'])
]

ct = ColumnTransformer(transformers, remainder='passthrough')
normalized_metadata_train = ct.fit_transform(metadata_train)

# Make dense array
normalized_metadata_train  = normalized_metadata_train.toarray()

In [48]:
pd.DataFrame(normalized_metadata_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.205340,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-1.358430,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,-0.797467,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-1.202607,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,-1.358430,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19596,0.137470,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
19597,7.398818,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
19598,-0.111847,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
19599,0.043976,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


## Initialize train dataloader ##

In [12]:
# TODO: do y
# normalized_waveform_train = torch.from_numpy(normalized_waveform_train)
# normalized_waveform_train = normalized_waveform_train.permute(0, 2, 1)
# print(normalized_waveform_train.shape)

# normalized_metadata_train = torch.tensor(normalized_metadata_train, dtype=torch.float32)
# print(normalized_metadata_train.shape)

# waveform_test = torch.tensor(waveform_test).float() 
# waveform_test = waveform_test.permute(0, 2, 1)

# metadata_test = torch.tensor(normalized_metadata_test, dtype=torch.float32)
# print(metadata_test.shape)

In [13]:
batch_size = 32
train_dataset = torch.from_numpy(normalized_waveform_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

## CNN autoencoder + LSTM metadata model ##

In [38]:
class CNNAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=12,
                      out_channels=128,
                      kernel_size=5,
                      stride=1),
            nn.MaxPool1d(kernel_size=2, 
                         stride=2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=128,
                               out_channels=12,
                               kernel_size=5,
                               stride=1),
            nn.Sigmoid(),
            nn.Upsample(size=1000)
        )
    
    def forward(self, x):
        encoded_output = self.encoder(x)
        decoded_output = self.decoder(encoded_output)
        return decoded_output

In [49]:
class MetadataLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(16, 32)
        self.output_layer = nn.Linear(32, 16)
    
    def forward(self, x, h):
        lstm_outputs, h_n = self.lstm(x, h)
        outputs = self.output_layer(lstm_outputs)
        return outputs, h_n

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, cnn_autoencoder, lstm_model, )

### Training ###

In [42]:
cnn_autoencoder_model = CNNAutoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(cnn_autoencoder_model.parameters(), lr=1e-3)

In [43]:
def train_cnn(model, train_loader, criterion, optimizer, nepoch=10):
    for epoch in range(nepoch):
        total_loss = 0
        for data in train_loader:
            inputs = data.permute(0, 2, 1).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{nepoch}, Loss: {total_loss/len(train_loader)}')

In [44]:
# Train the model
train_cnn(cnn_autoencoder_model, train_loader, criterion, optimizer)

Epoch 1/10, Loss: 0.004455086859064648
Epoch 2/10, Loss: 0.0014397300681825514
Epoch 3/10, Loss: 0.0012297008528798017
Epoch 4/10, Loss: 0.0011057490556042027
Epoch 5/10, Loss: 0.001025197899776555
Epoch 6/10, Loss: 0.000980121058014603
Epoch 7/10, Loss: 0.000935201107548644
Epoch 8/10, Loss: 0.0008989490218632239
Epoch 9/10, Loss: 0.0008759206383288526
Epoch 10/10, Loss: 0.000859810766664472


### Plotting ###

In [ ]:
def plot_waveform(waveform_df, segment_length=1000, start_index=0):
    plt.figure(figsize=(12, 6))
    for col in waveform_df.columns:
        plt.plot(waveform_df.index, waveform_df[col], label=f'Lead {col}')
    
    plt.title('ECG Waveform')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Amplitude')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.show()

## TCN Autoencoder ##

Model Card for the Hybrid Autoencoder Model Name: Hybrid Autoencoder for ECG and Metadata

Description: This model is designed to learn compressed representations of combined ECG waveform and patient metadata. It utilizes separate pathways for waveform data and metadata, merging them into a dense representation which is then used to reconstruct both types of data.

Model Architecture:

Waveform Pathway: Convolutional layers followed by pooling and flattening. Metadata Pathway: Dense layers. Combined Encoding and Decoding: Dense layers. Intended Use: Intended for anomaly detection in ECG data where additional patient metadata is available and considered relevant.

Data Used for Training: Assumes a dataset comprising ECG waveform data aligned with patient metadata such as age, sex, and device information.

Limitations: The model's effectiveness is highly dependent on the quality and preprocessing of the input data. The architecture needs fine-tuning and validation using real-world data to ensure robustness.

Ethical Considerations: Care should be taken to avoid biases that may arise from imbalanced data across different demographic groups. Privacy concerns should be addressed when handling patient data.

This framework sets up the foundation of your model; further tuning, training, and validation steps are needed to adapt it to specific tasks or datasets.

In [ ]:
from pytorch_tcn import TCN

class TCNAutoencoder(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout, metadata_dims):
        super(TCNAutoencoder, self).__init__()
        self.encoder = TCN(
            num_inputs=num_inputs,
            num_channels=num_channels,
            kernel_size=kernel_size,
            dropout=dropout,
            causal=True,
        )
        self.age_embedding = nn.Embedding(120, metadata_dims[0])  # Assuming age range from 0 to 119
        self.sex_embedding = nn.Embedding(2, metadata_dims[1])  # Assuming sex is binary (0 or 1)
        self.device_embedding = nn.Embedding(num_devices, metadata_dims[2])  # num_devices is the number of unique devices
        
        decoder_input_dim = num_channels[-1] + sum(metadata_dims)
        self.decoder = TCN(
            num_inputs=decoder_input_dim,
            num_channels=num_channels[::-1],
            kernel_size=kernel_size,
            dropout=dropout,    
            causal=True,
            output_projection=num_inputs,
        )
        
    def forward(self, x, age, sex, device):
        encoded = self.encoder(x)
        
        age_emb = self.age_embedding(age)
        sex_emb = self.sex_embedding(sex)
        device_emb = self.device_embedding(device)
        
        metadata_emb = torch.cat([age_emb, sex_emb, device_emb], dim=-1)
        metadata_emb = metadata_emb.unsqueeze(2).expand(-1, -1, encoded.size(2))
        
        concatenated = torch.cat([encoded, metadata_emb], dim=1)
        decoded = self.decoder(concatenated)
        return decoded